## Section 0

### Section 0.1

First, lets load the libraries we need

In [1]:
import os
import re
import html
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from matching_helpers import normaliser,\
                             parse_datetime,\
                             read_raw_data,\
                             prepare_nhsspend,\
                             prepare_contractsfinder,\
                             org_counter,\
                             strip_html,\
                             unique_agg,\
                             process_dates,\
                             make_matches

### Section 0.2

Lets load the raw payments data we need

In [2]:
tqdm.pandas()
df_centgov = read_raw_data('centgov_data.csv', 'Contracts Finder')
df_nhs = read_raw_data('nhsspend_data.csv', 'NHSSpend')
df_contracts = read_raw_data('contractsfinder_data.csv', 'Contracts Finder')

### Section 0.3

Wrangle them a bit

In [3]:
df_centgov = df_centgov[['data_source', 'amount', 'supplier', 'date', 'dept']]

In [4]:
df_nhs = prepare_nhsspend(df_nhs)

In [5]:
df_contracts = prepare_contractsfinder(df_contracts)

### Section 0.4

Merge and clean them a bit

In [ ]:
df_comb = pd.concat([df_nhs, df_centgov, df_contracts], ignore_index=True)
df_comb = df_comb.rename({'supplier': 'SUPPLIER'}, axis=1)
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].str.upper().str.strip()

df_comb['SUPPLIER_NUMERIC'] = pd.to_numeric(df_comb['SUPPLIER'], errors='coerce')
print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data because of numeric suppliers')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb = df_comb[df_comb['SUPPLIER_NUMERIC'].isna()]
df_comb = df_comb.drop(columns='SUPPLIER_NUMERIC')

print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data because of NaN suppliers')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].progress_apply(strip_html)
print(f'Dropping {len(df_comb[df_comb["SUPPLIER"].isnull()])} rows of data after html parsing')
df_comb = df_comb[df_comb['SUPPLIER'].notnull()]
df_comb['date'] = df_comb['date'].astype(str).str.split('T').str[0]
df_comb['date'] = pd.to_datetime(df_comb['date'],
                                 format='mixed',
                                 errors='coerce')
df_comb['date'] = df_comb['date'].map(lambda x: x.strftime('%d-%m-%Y') if pd.notnull(x) else np.nan)
print(f'Dropping {len(df_comb[df_comb["date"].isnull()])} rows of data due to NaN dates')
df_comb = df_comb[df_comb['date'].notnull()]
print(f'Dropping {len(df_comb[df_comb["amount"].isnull()])} rows of data due to NaN amounts')
df_comb = df_comb[df_comb['amount'].notnull()]
print(f'Dropping {len(df_comb[df_comb["dept"].isnull()])} rows of data due to NaN depts')
df_comb = df_comb[df_comb['dept'].notnull()]
df_comb['NORMALIZED_SUPPLIER'] = df_comb['SUPPLIER'].progress_apply(normaliser)
df_comb = df_comb[df_comb['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]

rows_to_drop = len(df_comb[
    (df_comb["SUPPLIER"].str.len() <= 3) |
    (df_comb["NORMALIZED_SUPPLIER"].str.len() <= 3)
])

# Print the message with the count of rows to be dropped
print(f'Dropping {rows_to_drop} rows of data due to supplier str len<=3')

df_comb = df_comb[
    (df_comb["SUPPLIER"].str.len() > 3) |
    (df_comb["NORMALIZED_SUPPLIER"].str.len() > 3)
]

df_comb[['SUPPLIER', 'ORG_COUNT']] = df_comb['SUPPLIER'].apply(lambda x: pd.Series(org_counter(x)))
all_rows = len(df_comb)

for supplier in ["SUCCESSFUL SUPPL",
                 "SEE ATTACH",
                 "REFER ATTACH",
                 "CONTRACT WAS AWARD",
                 "AWARDED SUPPLIERS",
                 "SUCCESSFUL SUPPLIER",
                 "PLEASE SEE",
                 'NAMED IND',
                 'REDACT',
                 "PLEASE REFER"]:
    df_comb = df_comb[~df_comb['SUPPLIER'].str.contains(supplier)]

print(f'Number of rows dropped due to redacted: {len(df_comb)-all_rows}')

print(f'Dropping {len(df_comb[df_comb["ORG_COUNT"]!=1])} where org_count !=1')
df_comb = df_comb[df_comb['ORG_COUNT']==1]

Dropping 926 rows of data because of numeric suppliers
Dropping 0 rows of data because of NaN suppliers


  0%|          | 0/9331286 [00:00<?, ?it/s]

### Section 0.5

Merge them into a unique dataframe, and then process it (inc. normalize)

In [ ]:
df_uniq = df_comb.pivot_table(index=['SUPPLIER'],
                              values=['date',
                                      'contractsfinder_region',
                                      'contractsfinder_awardedToVcse',
                                      'dept'],
                              aggfunc=unique_agg).reset_index()
df_sum = df_comb.groupby('SUPPLIER')['amount'].sum().reset_index()
df_counts = df_comb['SUPPLIER'].value_counts().reset_index()
df_uniq = pd.merge(df_uniq,
                   df_sum,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='SUPPLIER'
                  )
df_uniq[['SUPPLIER', 'ORG_COUNT']] = df_uniq['SUPPLIER'].progress_apply(lambda x: pd.Series(org_counter(x)))
df_uniq['NORMALIZED_SUPPLIER'] = df_uniq['SUPPLIER'].progress_apply(normaliser)
df_uniq = df_uniq[df_uniq['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]
df_uniq = pd.merge(df_uniq,
                   df_counts,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='SUPPLIER'
                  )
df_uniq.sort_values(by=['amount'],
                    ascending=False)

df_uniq1 = df_nhs[['supplier',
                   'NHSSpend_CompanyName',
                   'NHSSpend_CompanyNumber',
                   'NHSSpend_CharityName',
                   'NHSSpend_CharityRegNo',
                   'NHSSpend_CharitySubNo',
                   'NHSSpend_CharityNameNo',
                   'NHSSpend_CharityName']].drop_duplicates()
df_uniq = pd.merge(df_uniq,
                   df_uniq1,
                   how='left',
                   left_on='SUPPLIER',
                   right_on='supplier'
                  )


contractsfinder_region = df_sum = df_comb.groupby('SUPPLIER')['amount'].sum().reset_index()


df_uniq = df_uniq.rename({'count': 'PAYMENT_TOTAL_COUNT',
                          'amount': 'PAYMENT_TOTAL_AMOUNT'},
                         axis=1)
print(f'Dropping {len(df_uniq[df_uniq["ORG_COUNT"]!=1])} org_count !=1')
df_uniq = df_uniq[df_uniq['ORG_COUNT']==1]
df_uniq = df_uniq.drop(columns='supplier')
df_uniq['contractsfinder_awardedToVcse'] = df_uniq['contractsfinder_awardedToVcse'].apply(lambda x: "True" if True in x else "False")
df_uniq['deptcount'] = df_uniq['dept'].apply(len)
df_uniq['contractsfinder_region'] = df_uniq['contractsfinder_region'].apply(lambda x: "" if x == [np.nan] else x)
df_uniq['date'] = df_uniq['date'].astype(str).progress_apply(process_dates)
df_uniq['SUPPLIER'] = df_uniq['SUPPLIER'].replace('"', "[DQ]", regex=True)
df_uniq['dept'] = df_uniq['dept'].replace('"', "[DQ]", regex=True)
df_comb['SUPPLIER'] = df_comb['SUPPLIER'].replace('"', "[DQ]", regex=True)

### Section 0.6

Drop stuff that isn't any longer needed

In [ ]:
df_uniq = df_uniq.drop('NHSSpend_CharityNameNo', axis=1)
df_uniq = df_uniq.drop('NHSSpend_CharityName', axis=1)
df_uniq = df_uniq.drop('NHSSpend_CompanyName', axis=1)

df_comb = df_comb.drop('NHSSpend_CharityNameNo', axis=1)
df_comb = df_comb.drop('NHSSpend_CharityName', axis=1)
df_comb = df_comb.drop('NHSSpend_CompanyName', axis=1)
df_comb = df_comb.drop('NHSSpend_audit_type', axis=1)
df_comb = df_comb.drop('NHSSpend_CHnotes', axis=1)
df_comb = df_comb.drop('NHSSpend_CCnotes', axis=1)

### Section 0.7

See what's left in our dataframes

In [ ]:
print(df_uniq.columns)
print(df_uniq.head())

In [ ]:
print(df_comb.columns)
print(df_comb.head())

### Section 0.8

Sort, coerce, save uniq out before normalizing

In [ ]:
df_uniq = df_uniq.sort_values(by='PAYMENT_TOTAL_AMOUNT',
                              ascending=False)
df_uniq = df_uniq[df_uniq['NORMALIZED_SUPPLIER'].notnull()]
df_uniq = df_uniq[df_uniq['SUPPLIER'].notnull()]
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_raw.csv'),
               index=False)

In [ ]:
df_comb.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_all_raw.csv'),
               index=False
              )

# Section 1.0 

Lets now process the other auxillery registers

### Section 1.1

Lets load the uniq file back in so that we can split up the cleaning and matching process if we want to

### Section 1.1

CH first:

In [ ]:
df_ch = pd.read_csv(os.path.join(
    '..', 'registers', 'BasicCompanyDataAsOneFile-2024-08-01.csv'),
                    usecols=['CompanyName', ' CompanyNumber', 'RegAddress.PostTown', 'RegAddress.PostCode']
                )
df_ch['NORMALIZED_CompanyName'] = df_ch['CompanyName'].astype(str).progress_apply(normaliser)
df_ch = df_ch[df_ch['NORMALIZED_CompanyName'].apply(isinstance, args=(str,))]

df_ch = df_ch[df_ch['NORMALIZED_CompanyNama'].notnull()]
df_ch = df_ch.drop_duplicates(subset=['NORMALIZED_CompanyName'], keep=False)
df_ch = df_ch[[' CompanyNumber', 'NORMALIZED_CompanyName', 'CompanyName', 'RegAddress.PostTown', 'RegAddress.PostCode']]
df_ch['CompanyName'] = df_ch['CompanyName'].replace('"', "[DQ]", regex=True)
df_ch.to_csv(os.path.join('..', 'registers', 'ch_w_normalised.csv'), index=False)

### Section 1.2

Now the spine:

In [ ]:
df_spine = pd.read_csv(os.path.join(
    '..', 'registers', 'public_spine.spine.csv'),
                       usecols=['uid', 'organisationname', 'fulladdress', 'city', 'postcode', 'registerdate', 'removeddate'],
                       low_memory=False,
                )
df_spine['NORMALIZED_organisationname'] = df_spine['organisationname'].astype(str).progress_apply(normaliser)
df_spine = df_spine[df_spine['NORMALIZED_organisationname'].apply(isinstance, args=(str,))]
df_spine = df_spine[df_spine['NORMALIZED_organisationname'].notnull()]
df_spine = df_spine.drop_duplicates(subset=['NORMALIZED_organisationname'], keep=False)
df_spine = df_spine[['uid', 'NORMALIZED_organisationname', 'organisationname', 'fulladdress', 'city', 'postcode', 'registerdate', 'removeddate']]
df_spine['organisationname'] = df_spine['organisationname'].replace('"', "[DQ]", regex=True)
df_spine['fulladdress'] = df_spine['fulladdress'].replace('"', "[DQ]", regex=True)
df_spine.to_csv(os.path.join('..', 'registers', 'spine_w_normalised.csv'), index=False)

### Section 1.3

Now the NHSSpend NHS register:

In [ ]:
df_nhsdigital = pd.read_csv(os.path.join(
    '..', 'registers', 'nhs_uniq.csv')
                      )
df_nhsdigital['NORMALIZED_NHSDigital_Supplier'] = df_nhsdigital['NHSDigital_Supplier'].astype(str).progress_apply(normaliser)
df_nhsdigital = df_nhsdigital[df_nhsdigital['NORMALIZED_NHSDigital_Supplier'].apply(isinstance, args=(str,))]

df_nhsdigital = df_nhsdigital[df_nhsdigital['NORMALIZED_NHSDigital_Supplier'].notnull()]
df_nhsdigital = df_nhsdigital.drop_duplicates(subset=['NORMALIZED_NHSDigital_Supplier'], keep=False)
df_nhsdigital = df_nhsdigital[['NHSDigital_Supplier', 'NORMALIZED_NHSDigital_Supplier']]
df_nhsdigital['NHSDigital_Supplier'] = df_nhsdigital['NHSDigital_Supplier'].replace('"', "[DQ]", regex=True)
df_nhsdigital.to_csv(os.path.join('..', 'registers', 'nhsdigital_w_normalised.csv'), index=False)

# Section 2

### Section 2.1

Lets now load the files back in for the matching process.

#### Section 2.1.1

First, the uniq suppliers:

In [ ]:
df_uniq = pd.read_csv(os.path.join(os.getcwd(),
                                   '..',
                                   'raw_data',
                                   'merged_groupby_raw.csv')
                     )
print(f'We are then left with {len(df_uniq)} rows of unique "single" suppliers')

#### Section 2.1.2 

Then, the spine

In [ ]:
df_spine = pd.read_csv(os.path.join('..', 'registers', 'spine_w_normalised.csv'), low_memory=False)

#### Section 2.1.3

Then, the CH:

In [ ]:
df_ch = pd.read_csv(os.path.join('..', 'registers', 'ch_w_normalised.csv'))

#### Section 2.1.4

Then, the NHS Digital:

In [ ]:
df_nhsdigital = pd.read_csv(os.path.join('..', 'registers', 'nhsdigital_w_normalised.csv'))

# Section 3

Do the matches here.
    
#### Section 3.1.

Make the spine results

In [ ]:
df_uniq[~df_uniq['NORMALIZED_SUPPLIER'].notnull()]

In [ ]:
df_spine_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                                df_spine['NORMALIZED_organisationname'],
                                'spine')
df_spine_results.to_csv(os.path.join('..', 'matches', 'matches_to_spine.csv'))

In [ ]:
df_uniq['NORMALIZED_SUPPLIER'][~df_uniq['NORMALIZED_SUPPLIER'].apply(isinstance, args=(str,))]

#### Section 3.2.

Make the CH results

In [ ]:
df_ch_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'],
                             df_ch['NORMALIZED_CompanyName'],
                             'ch')
df_ch_results.to_csv(os.path.join('..', 'matches', 'matches_to_ch.csv'))

#### Section 3.3.

Make the NHS Digital results

In [ ]:
df_nhsdigital_results = make_matches(df_uniq['NORMALIZED_SUPPLIER'], df_nhsdigital['NORMALIZED_NHSDigital_Supplier'], 'nhsdigital')
df_nhsdigital_results.to_csv(os.path.join('..', 'matches', 'matches_to_nhsdigital.csv'))

# Section 4

Now, merge the matches back onto the unique.

### Section 4.1.

First, load the matches in

In [ ]:
df_spine_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_spine.csv'), index_col=0)
df_ch_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_ch.csv'), index_col=0)
df_nhsdigital_results = pd.read_csv(os.path.join('..', 'matches', 'matches_to_nhsdigital.csv'), index_col=0)
df_uniq = pd.read_csv(os.path.join(os.getcwd(),
                                   '..',
                                   'raw_data',
                                   'merged_groupby_raw.csv')
                     )

In [ ]:
df_ch_results

### Section 4.2.

Add new empty fields

In [ ]:
df_uniq['verified_normalized_spine_name'] = np.nan
df_uniq['verified_spine_uid'] = np.nan
df_uniq['verified_normalized_ch_name'] = np.nan
df_uniq['verified_ch_uid'] = np.nan
df_uniq['verified_nhsdigital_name'] = np.nan

In [ ]:
df_uniq = df_uniq.join(df_spine_results, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine.csv'),
               index=False)

In [ ]:
df_uniq = df_uniq.join(df_ch_results, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine_and_ch.csv'),
               index=False)

In [ ]:
df_uniq = df_uniq.join(df_nhsdigital_results, how='left')
df_uniq.to_csv(os.path.join(os.getcwd(),
                            '..',
                            'raw_data',
                            'merged_groupby_with_approximate_spine_and_ch_and_nhsdigital.csv'),
               index=False)

# Section 4

Do some EDA here which checks that the matches have been linked back in appropriately

In [ ]:
df_uniq[['SUPPLIER', 'NORMALIZED_SUPPLIER',
         'best_spine_match_1', 'best_spine_match_1_score',
         'best_ch_match_1', 'best_ch_match_1_score',
         'best_nhsdigital_match_1', 'best_nhsdigital_match_1_score']]